# Perform initial setup

In [1]:
from theano.sandbox import cuda
%matplotlib inline
import utils
reload(utils)
from utils import *
from __future__ import division, print_function
path = ""
batch_size = 64
validation_size = 0.2

Using Theano backend.


# Create a 20% validation set

Preserve the class imbalances in the training set

In [ ]:
%cd "train"
species = glob("*")
for d in species:
    os.mkdir("..\valid\"+d)
    images = glob(d+"\*")
    shuf = np.random.permutation(images)
    sample_size = int(round(len(images) * validation_size , 0))
    for i in range(sample_size): os.rename(shuf[i], '../valid/' + shuf[i])
%cd ..
%pwd

Read labels

In [3]:
(val_classes, trn_classes, val_labels , trn_labels, val_filenames, trn_filenames, test_filenames) = get_classes(path)

Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


# Build CNN Model

In [ ]:
vgg640 = Vgg16BN(360,640).model
vgg640.pop()
vgg640.input_shape(), vgg640.output_shape()
vgg640.compile(Adam(), 'categorical_crossentropy' , metrics = ['accuracy'])

# Get data in 360 X 640 format and augment it

Introduce random variations in the training data including rotation, width shift, zoom etc

In [4]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, 
                               shear_range=0.15, zoom_range = 0.1 , channel_shift_range = 10. , horizontal_flip = True)

In [5]:
batches = get_batches(path+'train', gen,shuffle=False, batch_size=1, class_mode=None, target_size=(360,640))
trn = np.concatenate([batches.next() for i in range(batches.nb_sample)])

batches = get_batches(path+'valid', shuffle=False, batch_size=1, class_mode=None, target_size=(360,640))
val = np.concatenate([batches.next() for i in range(batches.nb_sample)])

Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.


# Pre compute features

In [ ]:
trn_features = vgg640.predict(trn, batch_size = 32, verbose =1)
val_features = vgg640.predict(val, batch_size = 32, verbose =1)
del trn, val

Repeat for test data

In [ ]:
batches = get_batches(path+'test', shuffle=False, batch_size=1, class_mode=None, target_size=(360,640))
test = np.concatenate([batches.next() for i in range(batches.nb_sample)])
test_features = vgg640.predict(test, batch_size = 32, verbose =1)
del test

Save a copy or load it from disk, if available 

################################################    RESUME FROM HERE   ###################################################

In [ ]:
FIRST_TIME = False
if(FIRST_TIME):
    save_array(path+'val_features.dat', val_features)
    save_array(path+'trn_features.dat', trn_features)
    save_array(path+'test_features.dat', test_features)
else:
    val_features = load_array(path+'val_features.dat')
    trn_features = load_array(path+'trn_features.dat')
    test_features = load_array(path+'test_features.dat')

# Fit Model

Remove the final dense layer making this is a fully convolutional net

In [ ]:
conv_layers,_ = split_at(vgg640,Convolution_2D)
nf=128, p = 0.

Add four convolution layers

In [ ]:
def get_final_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(8,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]
myModel = Sequential(get_final_layers())
myModel.summary()

Train a couple of epochs with high learning rate

In [ ]:
myModel.compile(Adam(lr=0.001),loss='categorical_crossentropy' , metrics = ['accuracy'])
myModel.fit(trn_features, trn_labels, batch_size = batch_size, nb_epoch = 2 , validation_data = (val_features, val_labels))

Train another 6 epochs with low learning rate

In [ ]:
myModel.optimizer.lr=1e-4
myModel.fit(trn_features, trn_labels, batch_size = batch_size, nb_epoch = 6 , validation_data = (val_features, val_labels))
myModel.save_weights(path+'models/myModel.h5')
myModel.evaluate(val_features,val_labels)

# Make a Submission

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)
predictions = myModel.predict(test_features, batch_size = batch_size)
predictions_clipped = do_clip(predictions, 0.72)
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.head()
subm_name = path+'submission.csv'
submission.to_csv(subm_name, index=False)